In [9]:
import torch
import random

#生成需要的数据集和label对应的关系
def generat_data(true_R,num_example,b):
    #生成电流值 为0-1之间均匀分布的值
    I=torch.rand(num_example,1)
    U=torch.matmul(I,true_R)#U=IR
    b=torch.normal(0,0.1,U.shape)
    U+=b
    return I,U

def data_into_batch(I,U,batch_size):
    num_u=len(U)
    indices=list(range(num_u))
    #将索引的顺序打乱
    random.shuffle(indices)
    #根据batchsize的大小将样本分组
    for indice in range(0,num_u,batch_size):
        batch_indices=torch.tensor(
            indices[indice:min(indice+batch_size,num_u)]
        )
        yield I[batch_indices],U[batch_indices]
#定义欧姆定律的模型 b是一个误差
def model(I,R,b):
    return torch.matmul(I,R)+b

def loss(U_hat,U):
    return (U_hat-U.reshape(U.shape))**2/2

#随机梯度下降
def sgd(params,learning_rate,batch_size):
    with torch.no_grad():
        for param in params:
            param -=learning_rate*param.grad/batch_size
            param.grad.zero_()

#这是我们生成数据的时候用到的R
true_R=torch.tensor([5.0,4.0,3.0,2.0,1.0]).reshape(1,-1)
true_b=1.0

I,U=generat_data(true_R,3000,true_b)
learning_rate=0.1
batch_size=30
num_epochs=6

#这是为我们需要寻找的R初始化
R=torch.normal(0,0.01,size=true_R.shape,requires_grad=True)
b=torch.zeros(1,requires_grad=True)

for epoch in range(num_epochs):
    for i,u, in  data_into_batch(I,U,batch_size):
        l =loss(model(i,R,b),u)
        l.sum().backward()
        sgd([R,b],learning_rate,batch_size)
    with torch.no_grad():
        train_loss=loss(model(I,R,b),U)
    print(f'epoch:{epoch+1} train_loss:{train_loss.mean()}\n')

print('TrueR=',true_R,'predicatedR=',R)

epoch:1 train_loss:0.07502570748329163

epoch:2 train_loss:0.02037825435400009

epoch:3 train_loss:0.008092350326478481

epoch:4 train_loss:0.005763792898505926

epoch:5 train_loss:0.005267137195914984

epoch:6 train_loss:0.005216540303081274

TrueR= tensor([[5., 4., 3., 2., 1.]]) predicatedR= tensor([[4.9756, 3.9753, 2.9758, 1.9752, 0.9719]], requires_grad=True)
